In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.3` // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
import org.joda.time.format.DateTimeFormat
import java.util.Properties
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.{DataFrame, SQLContext}
import org.apache.spark.{SparkConf, SparkContext}
import org.joda.time.{DateTime, Days}
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark.sql.SaveMode._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.SparkConf
import org.apache.spark.sql.SparkSession
import org.json4s._
import org.json4s.jackson.JsonMethods._
import org.joda.time.format.DateTimeFormat
import org.joda.time.{DateTime, Days}
val formatter = DateTimeFormat.forPattern("yyyy-MM-dd")
import scala.util.Try
import java.sql.{Connection, DriverManager, ResultSet}
import org.apache.spark.sql.functions._

import $ivy.$                                   // Or use any other 2.x version here
// import $ivy.`sh.almond::almond-spark:_` // Added automatically on importing Spark


import org.apache.spark.sql._


import org.apache.log4j.{Level, Logger}

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

import org.joda.time.format.DateTimeFormat

import java.util.Properties

import org.apache.spark.sql.DataFrame

import org.apache.spark.sql.{DataFrame, SQLContext}

import org.apache.spark.{SparkConf, SparkContext}

import org.joda.time.{DateTime, Days}

import org.apache.spark.sql.{DataFrame, SparkSession}

import org.apache.spark.sql.SaveMode._

import org.apache.spark.sql.expressions.Window

import org.apache.spark.SparkConf

import org.apache.spark.sql.SparkSession

import org.json4s._

import org.json4s.jackson.JsonMethods._

import org.joda.time.format.DateTimeFormat

import org.joda.time.{DateTime, Days}

formatter: org.joda.time.format.DateTimeFormatter = 

In [2]:
val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


spark: SparkSession = org.apache.spark.sql.SparkSession@e2c5d1f

In [3]:
def sc = spark.sparkContext

defined function sc

In [4]:
var df = spark.read.format("csv").option("inferSchema", "true").option("header", "true").load("Data.csv")

df: DataFrame = [country_code: string, device_id: string ... 12 more fields]

In [5]:
df.printSchema

root
 |-- country_code: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- advertising_id: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- platform: string (nullable = true)
 |-- session_id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timezone: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- app: string (nullable = true)
 |-- user_id: long (nullable = true)
 |-- v: string (nullable = true)
 |-- version: string (nullable = true)



In [17]:
// TASK 1 Calculate first 10 event after editor_open (hint:lag lead window functions)

// sort data by id and date. lag 10 times and save those in 10 new columns and filter out nulls
// after 10 lags, filter only the "editor_open" lines and implement collectList.
// count the number of each event and take top 10 events

val top10 = df.select("device_id", "timestamp", "event_type").sort("device_id", "timestamp").toDF()

top10: DataFrame = [device_id: string, timestamp: timestamp ... 1 more field]

In [13]:
top10.agg(
    countDistinct("device_id")).take(1)(0)

res12: Row = [759578]

In [73]:
top10.registerTempTable("table")
val top10_01 = spark.sql("select device_id, timestamp, event_type, lag(event_type, 1) over (partition by device_id order by timestamp) as lag_1 from table")
top10_01.show(10)
                                                                                                                                                                        


+--------------------+-------------------+------------------+-----+
|           device_id|          timestamp|        event_type|lag_1|
+--------------------+-------------------+------------------+-----+
|00212dfa-e9ae-427...|2018-02-03 16:34:05|       editor_open| null|
|00291a51-155e-4c6...|2018-03-01 17:28:41|     shop_response| null|
|002d335f-8607-4bb...|2018-03-02 18:04:14|     shop_response| null|
|0037795f-9706-414...|2018-03-01 15:57:40|     shop_response| null|
|003f1ae7-8e7b-490...|2018-02-05 07:04:40|native_ad_response| null|
|0055d8ea-05d7-4fb...|2018-02-21 11:25:54|banner_ad_response| null|
|00652f6e-bb76-45a...|2018-02-11 07:10:58| banner_ad_request| null|
|008e7a4e-209f-445...|2018-02-23 16:52:22|     sticker_close| null|
|0091B3D0-00FD-46E...|2018-02-22 15:05:43|     shop_response| null|
|00B6DB90-F753-481...|2018-02-05 14:25:44|   tool_crop_close| null|
+--------------------+-------------------+------------------+-----+
only showing top 10 rows



top10_01: DataFrame = [device_id: string, timestamp: timestamp ... 2 more fields]

In [84]:
top10_01.registerTempTable("table1")
val top10_02 = (spark.sql("select device_id, timestamp, event_type, lag_1, lag(lag_1, 1) over (partition by device_id order by timestamp) as lag_2 from table1")).filter(col("lag_2").isNotNull)
top10_02.show(10)




+--------------------+-------------------+--------------------+--------------------+--------------------+
|           device_id|          timestamp|          event_type|               lag_1|               lag_2|
+--------------------+-------------------+--------------------+--------------------+--------------------+
|2a9d2378-2ce6-405...|2018-02-24 21:18:22|edit_top_menu_ite...|          photo_view|          photo_view|
|307b8a92-69cc-46f...|2018-02-20 15:12:48|        shop_request|          photo_view|          photo_view|
|320abc0a-a0c3-4ce...|2018-02-24 15:36:33|  editor_ad_response|      edit_item_open|picsart_share_pag...|
|675b51bd-8417-4e3...|2018-02-18 02:17:40|     tool_crop_close|photo_chooser_mul...|   tags_widget_shown|
|8488b92f-c48a-4ac...|2018-02-07 12:56:49|           card_view|           card_view|   native_ad_request|
|875f23e3-8cfc-445...|2018-02-26 16:11:47|  banner_ad_response|        sticker_view|           card_view|
|875f23e3-8cfc-445...|2018-03-02 14:03:28|rewa

top10_02: Dataset[Row] = [device_id: string, timestamp: timestamp ... 3 more fields]

In [86]:
top10_02.registerTempTable("table2")
val top10_03 = (spark.sql("select device_id, timestamp, event_type, lag_1, lag_2, lag(lag_2, 1) over (partition by device_id order by timestamp) as lag_3 from table2")).filter(col("lag_3").isNotNull)
top10_03.show(10)

+--------------------+-------------------+--------------------+--------------------+--------------------+------------------+
|           device_id|          timestamp|          event_type|               lag_1|               lag_2|             lag_3|
+--------------------+-------------------+--------------------+--------------------+--------------------+------------------+
|875f23e3-8cfc-445...|2018-03-02 14:03:28|rewarded_video_ad...|  banner_ad_response|        sticker_view|         card_view|
|3f46b1ce-b579-414...|2018-02-19 03:11:55|        shop_request|quick_brush_page_...|interstitial_ad_r...|banner_ad_response|
|c7f44f92-e581-47c...|2018-02-25 15:54:36|  banner_ad_response|          photo_view|         photo_swipe|       photo_close|
|1a428e36-0a2a-4ac...|2018-02-28 11:33:12|   native_ad_request|          photo_view|           card_view|        photo_open|
|b7b7488a-f32a-49b...|2018-02-19 11:20:55|   editor_done_click|  smash_widget_shown|        photo_upload|banner_ad_response|


top10_03: Dataset[Row] = [device_id: string, timestamp: timestamp ... 4 more fields]